In [5]:
import os
import google.generativeai as genai
import json

In [6]:
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

KeyError: 'GOOGLE_API_KEY'

In [3]:
EXAM_SAMPLE = '''\
Soit f(x) = x^3 - 3x^2 + 5x - 2.
Calculer f'(x). (6 points)
Expliquer les regles utilisees pour obtenir la derivee (regle de puissance, linearite). (4 points)
Resoudre l'equation f'(x) = 0 et donner les points critiques. (5 points)
Calculer f'(2) et donner une interpretation. (5 points)"
'''

In [4]:
STUDENT_SAMPLE = '''\
La derivee est f'(x) = 3x^2 - 6x + 5.

J'ai applique la regle de puissance: (x^3)' = 3x^2 et (-3x^2)' = -3x (je pense que c'est correct).
Ensuite par linearite j'ai juste additionne toutes les derivees.
(5x)' = 5 et (-2)' = -2 mais je pense qu'on le garde pas dans la derivee.

Pour resoudre f'(x) = 0:
3x^2 - 6x + 5 = 0.

J'ai calcule le discriminant: Delta = (-6)^2 - 435 = 36 - 60 = 24, donc Delta > 0 donc il y a 2 solutions reelles:
x = (6 +/- sqrt(24)) / 6.

Donc les points critiques sont environ x1 = 1 - sqrt(6)/3 et x2 = 1 + sqrt(6)/3.

Ensuite j'ai calcule f'(2):
f'(2) = 34 - 62 + 5 = 12 - 12 + 5 = 5.
Donc la pente en x=2 est 5.
'''

In [ ]:
def extract_rubric(self, exam_text):
    """
    Step 1: Extract the grading criteria (Barème) from the exam paper.
    """
    prompt = f"""
    Act as an expert pedagogical engineer. Analyze the following Exam Paper.
    Extract the grading rubric into a structured JSON format.
    For each question or section, identify:
    - The question ID/Number.
    - The topic.
    - The maximum points assigned.
    - The key elements expected in the answer (keywords, concepts).

    EXAM PAPER:
    {exam_text}
    """

    # Enforce JSON output schema
    result = self.model.generate_content(
        prompt,
        generation_config={"response_mime_type": "application/json"}
    )
    return json.loads(result.text)

In [ ]:
def grade_student(self, rubric_json, student_copy):
    """
    Step 2: Grade the student copy based ONLY on the extracted rubric.
    """
    prompt = f"""
    Act as a strict but fair academic grader.

    INPUT DATA:
    1. RUBRIC (JSON): {json.dumps(rubric_json)}
    2. STUDENT COPY: {student_copy}

    INSTRUCTIONS:
    - Go through the rubric item by item.
    - Compare the student's answer to the "expected elements".
    - Assign a score for each item (do not exceed max points).
    - Provide a justification for the score (mention missing keywords or logic errors).
    - Calculate the final total score.

    Output valid JSON.
    """

    result = self.model.generate_content(
        prompt,
        generation_config={"response_mime_type": "application/json"}
    )
    return json.loads(result.text)